In [3]:
import os
import numpy as np
import pandas as pd
from libemg.data_handler import OfflineDataHandler
import json
import sys
np.set_printoptions(threshold=sys.maxsize)

In [1]:
import mne

# Load the file
file_path = 'raw_data/eeg-motor-movement/S001/S001R03.edf'
raw = mne.io.read_raw_edf(file_path, preload=True)

# Print basic details
print(raw.info)
print("Channels:", raw.ch_names)
print("Sampling Rate (Hz):", raw.info['sfreq'])
print("Duration (s):", raw.n_times / raw.info['sfreq'])

# To get the data
data, times = raw[:]
print(data.shape)
print(times.shape)

Extracting EDF parameters from c:\Users\jrath\Documents\discrete-hand-gesture-recognition\raw_data\eeg-motor-movement\S001\S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
 subject_info: 3 items (dict)
>
Channels: ['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', '

In [2]:
# Load the EDF file
raw = mne.io.read_raw_edf(file_path, preload=False, verbose=False)

# Access annotations
annotations = raw.annotations

# Print annotations
print("Annotations in the file:")
for desc, onset, duration in zip(annotations.description, annotations.onset, annotations.duration):
    print(f"Description: {desc}, Onset: {onset}, Duration: {duration}")

# Convert annotations to events
events, event_id = mne.events_from_annotations(raw)

# Print events
print("\nEvent Codes and Corresponding Times:")
print(events)

# Print event ID mapping
print("\nEvent ID Mapping:")
print(event_id)

Annotations in the file:
Description: T0, Onset: 0.0, Duration: 4.2
Description: T2, Onset: 4.2, Duration: 4.1
Description: T0, Onset: 8.3, Duration: 4.2
Description: T1, Onset: 12.5, Duration: 4.1
Description: T0, Onset: 16.6, Duration: 4.2
Description: T1, Onset: 20.8, Duration: 4.1
Description: T0, Onset: 24.9, Duration: 4.2
Description: T2, Onset: 29.1, Duration: 4.1
Description: T0, Onset: 33.2, Duration: 4.2
Description: T2, Onset: 37.4, Duration: 4.1
Description: T0, Onset: 41.5, Duration: 4.2
Description: T1, Onset: 45.7, Duration: 4.1
Description: T0, Onset: 49.8, Duration: 4.2
Description: T1, Onset: 54.0, Duration: 4.1
Description: T0, Onset: 58.1, Duration: 4.2
Description: T2, Onset: 62.3, Duration: 4.1
Description: T0, Onset: 66.4, Duration: 4.2
Description: T1, Onset: 70.6, Duration: 4.1
Description: T0, Onset: 74.7, Duration: 4.2
Description: T2, Onset: 78.9, Duration: 4.1
Description: T0, Onset: 83.0, Duration: 4.2
Description: T2, Onset: 87.2, Duration: 4.1
Descriptio

In [3]:
import pandas as pd
raw = mne.io.read_raw_edf(file_path, preload=False, verbose=False)
annotations = raw.annotations
# Sampling frequency
sfreq = raw.info['sfreq']
# Prepare metadata list
metadata = []
# Loop through annotations and calculate start and end indices
for desc, onset, duration in zip(annotations.description, annotations.onset, annotations.duration):
    start_idx = int(onset * sfreq)
    end_idx = int((onset + duration) * sfreq)
    metadata.append({
        "Label": desc,
        "Start Index": start_idx,
        "End Index": end_idx
    })
# Convert to a structured DataFrame
metadata_df = pd.DataFrame(metadata)
metadata_df

,Label,Start Index,End Index
0,T0,0,672
1,T2,672,1328
2,T0,1328,2000
3,T1,2000,2656
4,T0,2656,3328
5,T1,3328,3984
6,T0,3984,4656
7,T2,4656,5312
8,T0,5312,5984
9,T2,5984,6640


In [4]:
entry = metadata[27]
entry

{'Label': 'T2', 'Start Index': 17936, 'End Index': 18592}

In [19]:
import time
start_time = time.time()
raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)

# Extract raw data for the given indices
start_idx, end_idx = entry["Start Index"], entry["End Index"]
raw_data_segment, times = raw[:, start_idx:end_idx]
# Create a dictionary with the segment and corresponding metadata
raw_data_info = {
    "Label": entry["Label"],
    "Start Index": start_idx,
    "End Index": end_idx,
    "Raw Data Shape": raw_data_segment.shape
}


print(f"File read took {time.time() - start_time:.4f} seconds")

File read took 0.0126 seconds


In [31]:
channel_groups = {
            "Frontal_Left": ['Fp1.', 'Af7.', 'F7..', 'F5..', 'F3..', 'F1..'],
            "Frontal_Right": ['Fp2.', 'Af8.', 'F8..', 'F6..', 'F4..', 'F2..'],
            "Central": ['Cz..', 'Fc1.', 'Fc2.', 'C1..', 'C2..'],
            "Temporal_Left": ['Ft7.', 'T7..', 'Tp7.', 'T9..'],
            "Temporal_Right": ['Ft8.', 'T8..', 'Tp8.', 'T10.'],
            "Parietal_Left": ['P7..', 'P5..', 'P3..', 'P1..', 'Po7.', 'Po3.'],
            "Parietal_Right": ['P8..', 'P6..', 'P4..', 'P2..', 'Po8.', 'Po4.'],
            "Occipital": ['O1..', 'Oz..', 'O2..', 'Poz.', 'Iz..']
        }

In [33]:
def combine_channels(eeg_data, channel_names):
    """
    Combine 64 channels into 8 regions by averaging grouped channels.
    
    :param eeg_data: Numpy array of shape (n_channels, n_times)
    :param channel_names: List of channel names in the dataset
    :return: Numpy array of shape (8, n_times)
    """
    # Initialize an array to hold the reduced 8 channels
    reduced_data = np.zeros((len(channel_groups), eeg_data.shape[1]))
    #print(channel_names)
    for i, (region, channels) in enumerate(channel_groups.items()):
        # Find indices of channels belonging to this region
        indices = [channel_names.index(ch) for ch in channels if ch in channel_names]

        # Check if channels for this region exist
        if len(indices) > 0:
            # Average the signals across the selected channels
            reduced_data[i, :] = np.mean(eeg_data[indices, :], axis=0)
        else:
            print(f"Warning: No channels found for region {region}. Filling with zeros.")
    
    return reduced_data

In [43]:
import numpy as np
raw = mne.io.read_raw_edf(file_path, preload=False, verbose=False)
raw_data_segment, times = raw[:, start_idx:end_idx]

# Convert the data to numpy array
eeg_data = np.array(raw_data_segment)

# Combine channels into 8 regions
region_data = combine_channels(eeg_data, raw.info['ch_names'])
#region_data.dtype
eeg_data.dtype

dtype('float64')

In [4]:
import h5py
import json
import numpy as np

# Convert JSON to HDF5
with open("raw_data/EMG-EPN612/trainingJSON/user1/user1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def save_dict_to_hdf5(h5f, dictionary, path=""):
    """
    Recursively saves a dictionary to an HDF5 file.
    :param h5f: HDF5 file object
    :param dictionary: The dictionary to save
    :param path: The path in the HDF5 file
    """
    for key, value in dictionary.items():
        current_path = f"{path}/{key}" if path else key
        if current_path in h5f:
            # Skip if group/dataset already exists
            print(f"Skipping {current_path}, already exists.")
            continue
        if isinstance(value, dict):
            # Create a group for nested dictionaries
            group = h5f.create_group(current_path)
            save_dict_to_hdf5(group, value)  # Recurse
        elif isinstance(value, list):
            # Convert lists to NumPy arrays
            value = np.array(value)
            h5f.create_dataset(current_path, data=value)
        elif isinstance(value, str):
            # Convert strings to fixed-length format
            value = np.string_(value)
            h5f.create_dataset(current_path, data=value)
        else:
            # Directly save other types (e.g., numerical arrays)
            h5f.create_dataset(current_path, data=value)

with h5py.File("example.h5", "w") as h5f:
    for key, value in data.items():
        
        save_dict_to_hdf5(h5f, data)

Skipping generalInfo, already exists.
Skipping userInfo, already exists.
Skipping synchronizationGesture, already exists.
Skipping trainingSamples, already exists.
Skipping testingSamples, already exists.
Skipping generalInfo, already exists.
Skipping userInfo, already exists.
Skipping synchronizationGesture, already exists.
Skipping trainingSamples, already exists.
Skipping testingSamples, already exists.
Skipping generalInfo, already exists.
Skipping userInfo, already exists.
Skipping synchronizationGesture, already exists.
Skipping trainingSamples, already exists.
Skipping testingSamples, already exists.
Skipping generalInfo, already exists.
Skipping userInfo, already exists.
Skipping synchronizationGesture, already exists.
Skipping trainingSamples, already exists.
Skipping testingSamples, already exists.


In [5]:
import time

file_path = 'raw_data/EMG-EPN612/trainingJSON/user1/user1.json'
start_time = time.time()
with h5py.File(file_path.replace(".json", ".h5"), "r") as f:
        sample_data = f["trainingSamples"]['idx_1']
        if "gestureName" in sample_data:
            startingPoint = 0
            if sample_data["gestureName"] != "noGesture":
                raw_starting_point = sample_data.get("startPointforGestureExecution", 0)
                if isinstance(raw_starting_point, (int, float)):
                        startingPoint = int(raw_starting_point)
                elif hasattr(raw_starting_point, "__getitem__"):  # Handle HDF5 datasets
                        startingPoint = int(raw_starting_point[()])  # Extract scalar value
                else:
                        raise TypeError(f"Unexpected type for startingPoint: {type(raw_starting_point)}")
            emg_data = np.array([
                sample_data["emg"]["ch1"][startingPoint:],
                sample_data["emg"]["ch2"][startingPoint:],
                sample_data["emg"]["ch3"][startingPoint:],
                sample_data["emg"]["ch4"][startingPoint:],
                sample_data["emg"]["ch5"][startingPoint:],
                sample_data["emg"]["ch6"][startingPoint:],
                sample_data["emg"]["ch7"][startingPoint:],
                sample_data["emg"]["ch8"][startingPoint:]
            ])   
print(f"File read took {time.time() - start_time:.4f} seconds")

File read took 0.0181 seconds


In [44]:
# Loop 10 times
def json(loops):
    import json
    execution_times = []
    for _ in range(loops):
        start_time = time.time()
        with open(file_path, 'r', encoding='utf-8') as f:
                    user_data = json.load(f)
                    if "trainingSamples" in user_data:
                        sample_data = user_data["trainingSamples"]['idx_1']
                        if "gestureName" in sample_data:
                            startingPoint = 0
                            if sample_data["gestureName"] != "noGesture":
                                startingPoint = sample_data["startPointforGestureExecution"]
                            emg_data = np.array([
                                sample_data["emg"]["ch1"][startingPoint:],
                                sample_data["emg"]["ch2"][startingPoint:],
                                sample_data["emg"]["ch3"][startingPoint:],
                                sample_data["emg"]["ch4"][startingPoint:],
                                sample_data["emg"]["ch5"][startingPoint:],
                                sample_data["emg"]["ch6"][startingPoint:],
                                sample_data["emg"]["ch7"][startingPoint:],
                                sample_data["emg"]["ch8"][startingPoint:]
                            ])   
        execution_times.append(time.time() - start_time)
    average_time = sum(execution_times) / len(execution_times)
    print(f"File read took {average_time:.4f} seconds")

In [45]:
def ujson(loops):    
    import ujson as json
    execution_times = []
    for _ in range(loops):
        start_time = time.time()

        with open(file_path, 'r', encoding='utf-8') as f:
                    user_data = json.load(f)
                    if "trainingSamples" in user_data:
                        sample_data = user_data["trainingSamples"]['idx_1']
                        if "gestureName" in sample_data:
                            startingPoint = 0
                            if sample_data["gestureName"] != "noGesture":
                                startingPoint = sample_data["startPointforGestureExecution"]
                            emg_data = np.array([
                                sample_data["emg"]["ch1"][startingPoint:],
                                sample_data["emg"]["ch2"][startingPoint:],
                                sample_data["emg"]["ch3"][startingPoint:],
                                sample_data["emg"]["ch4"][startingPoint:],
                                sample_data["emg"]["ch5"][startingPoint:],
                                sample_data["emg"]["ch6"][startingPoint:],
                                sample_data["emg"]["ch7"][startingPoint:],
                                sample_data["emg"]["ch8"][startingPoint:]
                            ])   
        execution_times.append(time.time() - start_time)
    average_time = sum(execution_times) / len(execution_times)
    print(f"File read took {average_time:.4f} seconds")

In [46]:
def orjson(loops):    
    import orjson as json
    execution_times = []
    for _ in range(loops):
        start_time = time.time()
        with open(file_path, 'r', encoding='utf-8') as f:
                    user_data = json.loads(f.read())
                    if "trainingSamples" in user_data:
                        sample_data = user_data["trainingSamples"]['idx_1']
                        if "gestureName" in sample_data:
                            startingPoint = 0
                            if sample_data["gestureName"] != "noGesture":
                                startingPoint = sample_data["startPointforGestureExecution"]
                            emg_data = np.array([
                                sample_data["emg"]["ch1"][startingPoint:],
                                sample_data["emg"]["ch2"][startingPoint:],
                                sample_data["emg"]["ch3"][startingPoint:],
                                sample_data["emg"]["ch4"][startingPoint:],
                                sample_data["emg"]["ch5"][startingPoint:],
                                sample_data["emg"]["ch6"][startingPoint:],
                                sample_data["emg"]["ch7"][startingPoint:],
                                sample_data["emg"]["ch8"][startingPoint:]
                            ])   
        execution_times.append(time.time() - start_time)
    average_time = sum(execution_times) / len(execution_times)
    print(f"File read took {average_time:.4f} seconds")

In [47]:
def msgspec(loops):    
    import msgspec.json
    execution_times = []
    for _ in range(loops):
        start_time = time.time()
        with open(file_path, "rb") as f:
                    user_data = msgspec.json.decode(f.read())
                    if "trainingSamples" in user_data:
                        sample_data = user_data["trainingSamples"]['idx_1']
                        if "gestureName" in sample_data:
                            startingPoint = 0
                            if sample_data["gestureName"] != "noGesture":
                                startingPoint = sample_data["startPointforGestureExecution"]
                            emg_data = np.array([
                                sample_data["emg"]["ch1"][startingPoint:],
                                sample_data["emg"]["ch2"][startingPoint:],
                                sample_data["emg"]["ch3"][startingPoint:],
                                sample_data["emg"]["ch4"][startingPoint:],
                                sample_data["emg"]["ch5"][startingPoint:],
                                sample_data["emg"]["ch6"][startingPoint:],
                                sample_data["emg"]["ch7"][startingPoint:],
                                sample_data["emg"]["ch8"][startingPoint:]
                            ])   
        execution_times.append(time.time() - start_time)
    average_time = sum(execution_times) / len(execution_times)
    print(f"File read took {average_time:.4f} seconds")

In [50]:
def msgspec_adv(loops):    
    import msgspec.json
    execution_times = []
    for _ in range(loops):
        start_time = time.time()
        with open(file_path, "rb") as f:
            user_data = msgspec.json.decode(f.read())
        if "trainingSamples" in user_data:
            sample_data = user_data["trainingSamples"]['idx_1']
            if "gestureName" in sample_data:
                startingPoint = 0
                if sample_data["gestureName"] != "noGesture":
                    startingPoint = sample_data["startPointforGestureExecution"]
                emg_data = np.array([
                    sample_data["emg"]["ch1"][startingPoint:],
                    sample_data["emg"]["ch2"][startingPoint:],
                    sample_data["emg"]["ch3"][startingPoint:],
                    sample_data["emg"]["ch4"][startingPoint:],
                    sample_data["emg"]["ch5"][startingPoint:],
                    sample_data["emg"]["ch6"][startingPoint:],
                    sample_data["emg"]["ch7"][startingPoint:],
                    sample_data["emg"]["ch8"][startingPoint:]
                ])   
        execution_times.append(time.time() - start_time)
    average_time = sum(execution_times) / len(execution_times)
    print(f"File read took {average_time:.4f} seconds")

In [48]:
print("json:")
json(50)
print("orjson:")
orjson(50)
print("ujson:")
ujson(50)
print("msgspec:")
msgspec(50)

json:
File read took 0.2490 seconds
orjson:
File read took 0.0953 seconds
ujson:
File read took 0.1082 seconds
msgspec:
File read took 0.0863 seconds


In [51]:
print("msgspec:")
msgspec(50)
print("msgspec_adv:")
msgspec_adv(50)

msgspec:
File read took 0.0878 seconds
msgspec_adv:
File read took 0.0867 seconds


In [55]:
import msgspec.json


start_time = time.time()
with open(file_path, "rb") as f:
            user_data = msgspec.json.decode(f.read())
            file_open = (time.time() - start_time)
            start_time = time.time()
            if "trainingSamples" in user_data:
                sample_data = user_data["trainingSamples"]['idx_1']
                if "gestureName" in sample_data:
                    startingPoint = 0
                    if sample_data["gestureName"] != "noGesture":
                        startingPoint = sample_data["startPointforGestureExecution"]
                    emg_data = np.array([
                        sample_data["emg"]["ch1"][startingPoint:],
                        sample_data["emg"]["ch2"][startingPoint:],
                        sample_data["emg"]["ch3"][startingPoint:],
                        sample_data["emg"]["ch4"][startingPoint:],
                        sample_data["emg"]["ch5"][startingPoint:],
                        sample_data["emg"]["ch6"][startingPoint:],
                        sample_data["emg"]["ch7"][startingPoint:],
                        sample_data["emg"]["ch8"][startingPoint:]
                    ])   
            gesture_read = (time.time() - start_time)

print(f"File read took {file_open:.4f} seconds")
print(f"File read took {gesture_read:.4f} seconds")

File read took 0.0823 seconds
File read took 0.0000 seconds


In [1]:
import pandas as pd

df = pd.read_csv('raw_data/EMG-EPN612/training-metadata.csv')

# Group data by File_Path
grouped = df.groupby("File_Path")  
grouped.head()

,Label,File_Path,Gesture_Index
0,noGesture,raw_data/EMG-EPN612/trainingJSON/user1/user1.json,idx_1
1,noGesture,raw_data/EMG-EPN612/trainingJSON/user1/user1.json,idx_2
2,noGesture,raw_data/EMG-EPN612/trainingJSON/user1/user1.json,idx_3
3,noGesture,raw_data/EMG-EPN612/trainingJSON/user1/user1.json,idx_4
4,noGesture,raw_data/EMG-EPN612/trainingJSON/user1/user1.json,idx_5
...,...,...,...
45750,noGesture,raw_data/EMG-EPN612/trainingJSON/user99/user99...,idx_1
45751,noGesture,raw_data/EMG-EPN612/trainingJSON/user99/user99...,idx_2
45752,noGesture,raw_data/EMG-EPN612/trainingJSON/user99/user99...,idx_3
45753,noGesture,raw_data/EMG-EPN612/trainingJSON/user99/user99...,idx_4


In [57]:
import json

def write_h5(group, data):
    """
    Recursively write JSON-like data into an HDF5 group.
    """
    for key, value in data.items():
        if isinstance(value, dict):  # Nested structure, create a subgroup
            subgroup = group.create_group(key)
            write_h5(subgroup, value)
        elif isinstance(value, list):  # Handle lists as datasets
            group.create_dataset(key, data=value)
        elif isinstance(value, (int, float, str)):  # Handle scalar values
            group.attrs[key] = value
        else:
            raise TypeError(f"Unsupported data type for key: {key}, value: {value}")
        
json_file = 'raw_data/EMG-EPN612/trainingJSON/user1/user1.json'
h5_file = 'raw_data/EMG-EPN612/trainingJSON/user1/user1.h5'

# Load JSON and write to HDF5
with open(json_file, 'r') as f:
    json_data = json.load(f)

with h5py.File(h5_file, 'w') as h5f:
    write_h5(h5f, json_data)

print(f"Data successfully written to {h5_file}")


Data successfully written to raw_data/EMG-EPN612/trainingJSON/user1/user1.h5


In [106]:
import h5py
import numpy as np

# Open the HDF5 file
file_path = 'raw_data/EMG-EPN612/trainingJSON/user1/user1.h5'
start_time = time.time()
with h5py.File(file_path, 'r') as h5f:
    # Access the EMG group
    sample_data = h5f['trainingSamples/idx_1/']
    gestureName = sample_data.attrs['gestureName']
    emg_group = h5f['trainingSamples/idx_1/emg']
    print(gestureName)
    if gestureName != 'noGesture':
        startingPoint = 0
        raw_starting_point = sample_data.attrs.get("startPointforGestureExecution", 0)
        if isinstance(raw_starting_point, (int, float)):
                        startingPoint = int(raw_starting_point)
        elif hasattr(raw_starting_point, "__getitem__"):  # Handle HDF5 datasets
                startingPoint = int(raw_starting_point[()])  # Extract scalar value
    # Load all channels into a list
    channels = []
    for channel_name in emg_group.keys():  # Iterate over 'ch1', 'ch2', ...
        channel_data = emg_group[channel_name][startingPoint:]
        channels.append(channel_data)
    
    # Convert the list to a NumPy array
    # Shape: (number_of_channels, number_of_samples)
    emg_array = np.array(channels)
    print((time.time() - start_time))

print(f"EMG data shape: {emg_array.shape}")
print(f"EMG data: {emg_array}")

noGesture
0.0014967918395996094
EMG data shape: (8, 992)
EMG data: [[-1  2 -1 ... -1 -1  0]
 [-1  2  1 ...  0  3  4]
 [ 2  0 -1 ... -5  0  0]
 ...
 [ 1 -2 -1 ...  0 -3  3]
 [-1  0 -1 ...  0 -4 -1]
 [ 0 -1  0 ... -1 -2 -1]]


In [107]:
file_path = 'raw_data/EMG-EPN612/trainingJSON/user1/user1.json'
with open(file_path, 'r', encoding='utf-8') as f:
                    user_data = json.load(f)
                    if "trainingSamples" in user_data:
                        sample_data = user_data["trainingSamples"]['idx_1']
                        if "gestureName" in sample_data:
                            startingPoint = 0
                            if sample_data["gestureName"] != "noGesture":
                                startingPoint = sample_data["startPointforGestureExecution"]
                            emg_data = np.array([
                                sample_data["emg"]["ch1"][startingPoint:],
                                sample_data["emg"]["ch2"][startingPoint:],
                                sample_data["emg"]["ch3"][startingPoint:],
                                sample_data["emg"]["ch4"][startingPoint:],
                                sample_data["emg"]["ch5"][startingPoint:],
                                sample_data["emg"]["ch6"][startingPoint:],
                                sample_data["emg"]["ch7"][startingPoint:],
                                sample_data["emg"]["ch8"][startingPoint:]
                            ])   

print(f"EMG data shape: {emg_data.shape}")
print(f"EMG data: {emg_data}")

EMG data shape: (8, 992)
EMG data: [[-1  2 -1 ... -1 -1  0]
 [-1  2  1 ...  0  3  4]
 [ 2  0 -1 ... -5  0  0]
 ...
 [ 1 -2 -1 ...  0 -3  3]
 [-1  0 -1 ...  0 -4 -1]
 [ 0 -1  0 ... -1 -2 -1]]


In [67]:
def print_h5_structure(h5_file, indent=0):
    """
    Recursively print the structure of an HDF5 file.
    
    Args:
        h5_file: h5py File or Group object.
        indent: Current level of indentation for printing.
    """
    for key in h5_file.keys():
        item = h5_file[key]
        print(" " * indent + f"{key}: {'Group' if isinstance(item, h5py.Group) else 'Dataset'}")
        
        # Print attributes if present
        if hasattr(item, "attrs") and len(item.attrs) > 0:
            for attr_key, attr_value in item.attrs.items():
                print(" " * (indent + 2) + f"Attribute - {attr_key}: {attr_value}")
        
        # Recursively print groups
        if isinstance(item, h5py.Group):
            print_h5_structure(item, indent + 2)

# Example usage
h5_file_path = "raw_data/EMG-EPN612/trainingJSON/user1/user1.h5"

with h5py.File(h5_file_path, 'r') as h5f:
    print_h5_structure(h5f)

generalInfo: Group
  Attribute - deviceModel: Myo Armband
  Attribute - recordingTimeInSeconds: 5
  Attribute - repetitionsForSynchronizationGesture: 5
  Attribute - samplingFrequencyInHertz: 200
  myoPredictionLabel: Group
    Attribute - fist: 1
    Attribute - noGesture: 0
    Attribute - open: 4
    Attribute - pinch: 5
    Attribute - waveIn: 2
    Attribute - waveOut: 3
synchronizationGesture: Group
  samples: Group
    idx_1: Group
      Attribute - startPointforGestureExecution: 793
      accelerometer: Group
        x: Dataset
        y: Dataset
        z: Dataset
      emg: Group
        ch1: Dataset
        ch2: Dataset
        ch3: Dataset
        ch4: Dataset
        ch5: Dataset
        ch6: Dataset
        ch7: Dataset
        ch8: Dataset
      gyroscope: Group
        x: Dataset
        y: Dataset
        z: Dataset
      myoDetection: Dataset
      quaternion: Group
        w: Dataset
        x: Dataset
        y: Dataset
        z: Dataset
    idx_2: Group
      Attr

## EMGEPN612 DataHandler and Data Loading

In [4]:
from tqdm import tqdm
from scipy.fftpack import fft

class EMGEPN612TrainingDataHandler(OfflineDataHandler):
    def __init__(self, data_dir):
        super().__init__()
        self.data_dir = data_dir
        self.data = []
        self.images = []
        self.labels = []
        self.max_length = 0 #846 #1200
        self.min_length = 2000
        self.get_data()
        print("MINMAX")
        print(self.max_length)
        print(self.min_length)
        self.pad_all_data_reflective()
        self.extract_feature_windows()

    def get_data(self):
        gesture = 0 
        noGesture = 0
        maxNumbers= len(os.listdir(self.data_dir))
        with tqdm(total=maxNumbers, desc="Processing Users", unit="user") as pbar:
            for user_folder in os.listdir(self.data_dir):
                user_path = os.path.join(self.data_dir, user_folder)
                # Ensure we are reading a directory with the expected JSON structure
                if os.path.isdir(user_path):
                    json_file = os.path.join(user_path, f"{user_folder}.json")
                    if os.path.isfile(json_file):
                        with open(json_file, 'r', encoding='utf-8') as f:
                            user_data = json.load(f)
                            # Extract data from trainingSamples
                            if "trainingSamples" in user_data:
                                for sample_key, sample_data in user_data["trainingSamples"].items():
                                    if "gestureName" in sample_data:
                                        gesture = gesture + 1
                                        startingPoint = 0
                                        if sample_data["gestureName"] != "noGesture":
                                            startingPoint = sample_data["startPointforGestureExecution"]
                                    # Extract EMG data across channels ch1-ch8
                                        emg_data = np.array([
                                            sample_data["emg"]["ch1"][startingPoint:],
                                            sample_data["emg"]["ch2"][startingPoint:],
                                            sample_data["emg"]["ch3"][startingPoint:],
                                            sample_data["emg"]["ch4"][startingPoint:],
                                            sample_data["emg"]["ch5"][startingPoint:],
                                            sample_data["emg"]["ch6"][startingPoint:],
                                            sample_data["emg"]["ch7"][startingPoint:],
                                            sample_data["emg"]["ch8"][startingPoint:]
                                        ])
                                        sample_length = len(emg_data[0])
                                        if sample_length < 200:
                                            print(sample_data["gestureName"])
                                        if sample_data["gestureName"] != "noGesture":
                                            self.max_length = max(self.max_length, sample_length)
                                        self.min_length = min(self.min_length, sample_length)
                                        # Store the EMG data and corresponding gesture label
                                        self.data.append(emg_data)
                                        label = -1
                                        match sample_data["gestureName"]:
                                            case "noGesture": 
                                                label = 0
                                            case "fist": 
                                                label = 1
                                            case "waveIn": 
                                                label = 2
                                            case "waveOut": 
                                                label = 3
                                            case "open": 
                                                label = 4
                                            case "pinch": 
                                                label = 5
                                                
                                        self.labels.append(label)
                                    else :
                                        noGesture = noGesture + 1
                pbar.update(1)
        print("Ground truth available: " + str(gesture))
        print("No Ground truth available: " + str(noGesture))
                                
    def pad_all_data(self):
        # Pad all sequences to the max length found
        print("max length: " + str(self.max_length))
        padded_data = []
        with tqdm(total=len(self.data), desc="Padding Data", unit="item") as pbar:
            for emg_data in self.data:
                if len(emg_data[0]) < self.max_length:
                    padded= np.array([np.pad(x, (0, self.max_length - len(x)), 'constant') for x in emg_data])
                    padded_data.append(padded)
                else:
                    print("nopadding")
                    padded_data.append(emg_data)
                pbar.update(1)
        self.data = padded_data

    
    def pad_all_data_reflective(self):
        """
        Pads all sequences in self.data to the max length found, using reflective padding.

        Updates self.data with padded sequences.

        Assumes self.data is a list where each element is a sequence of arrays, and each array represents
        a time-series feature vector.

        Reflective padding is used to ensure that the added data is contextually relevant.
        """
        print("max length: " + str(self.max_length))
        padded_data = []
        
        with tqdm(total=len(self.data), desc="Padding Data", unit="item") as pbar:
            for emg_data, label in zip(self.data, self.labels):
                # Check if the data needs padding
                if len(emg_data[0]) < self.max_length:
                    # Apply reflective padding to each feature vector
                    padded = np.array([
                        np.pad(x, (0, self.max_length - len(x)), mode='reflect') for x in emg_data
                    ])
                    padded_data.append(padded)
                else:
                    if label != 0:
                        print("TRUNCATED DATA WITH LABEL: " + str(label))
                    truncated = np.array([x[:self.max_length] for x in emg_data])
                    padded_data.append(truncated)
                
                pbar.update(1)
        
        self.data = padded_data

    def extract_feature_windows(self, fs=200, window_size=1.0, overlap=0.5):
        """
        Extract features from a signal using sliding windows.
        Args:
            signal (numpy array): Input signal.
            fs (int): Sampling frequency in Hz (default: 200).
            window_size (float): Sliding window size in seconds (default: 1.0).
            overlap (float): Overlap size in seconds (default: 0.5).

        Returns:
            numpy array: Extracted features for each window.
        """
        data = []
        labels = []
        with tqdm(total=len(self.data), desc="Calculating Windows", unit="item") as pbar:
            for emg_data, label in zip(self.data, self.labels):
                num_channels, num_samples = emg_data.shape
                window_samples = int(window_size * fs)
                step_samples = int(window_samples * (1 - overlap))
                num_windows = (num_samples - window_samples) // step_samples + 1
                gesture_windows = []

                for i in range(num_windows):
                    start = i * step_samples
                    end = start + window_samples
                    segment = emg_data[:, start:end]  # Shape: [num_channels, window_samples]

                    # Extract features for each channel
                    window_features = []
                    for channel in range(num_channels):
                        channel_segment = segment[channel, :]

                        # Compute half-windows
                        half1 = channel_segment[:window_samples // 2]
                        half2 = channel_segment[window_samples // 2:]

                        # Half-window features
                        mean1, mean2 = np.mean(half1), np.mean(half2)
                        std1, std2 = np.std(half1), np.std(half2)
                        max1, max2 = np.max(half1), np.max(half2)
                        min1, min2 = np.min(half1), np.min(half2)

                        changes = [
                            mean2 - mean1,
                            std2 - std1,
                            max2 - max1,
                            min2 - min1,
                        ]

                        # Compute quarter-windows
                        quarter1 = channel_segment[:window_samples // 4]
                        quarter2 = channel_segment[window_samples // 4:window_samples // 2]
                        quarter3 = channel_segment[window_samples // 2:3 * window_samples // 4]
                        quarter4 = channel_segment[3 * window_samples // 4:]

                        means = [np.mean(quarter) for quarter in [quarter1, quarter2, quarter3, quarter4]]
                        maxs = [np.max(quarter) for quarter in [quarter1, quarter2, quarter3, quarter4]]
                        mins = [np.min(quarter) for quarter in [quarter1, quarter2, quarter3, quarter4]]

                        pairwise_differences = [
                            means[i] - means[j] for i in range(len(means)) for j in range(i + 1, len(means))
                        ] + [
                            maxs[i] - maxs[j] for i in range(len(maxs)) for j in range(i + 1, len(maxs))
                        ] + [
                            mins[i] - mins[j] for i in range(len(mins)) for j in range(i + 1, len(mins))
                        ]

                        # DFT features
                        fft_magnitudes = np.abs(fft(channel_segment))[:len(channel_segment) // 2]
                        top_10_freqs = np.argsort(fft_magnitudes)[-10:]
                        # Combine features for this channel
                        channel_features = changes + means + maxs + mins + pairwise_differences + list(fft_magnitudes[top_10_freqs])
                        window_features.extend(channel_features)
                    gesture_windows.append(np.array(window_features))
                data.append(np.array(gesture_windows))
                    
                
                pbar.update(1)
            print(np.array(self.data).shape)
            self.data = data
            
            print(np.array(self.data).shape)

    def create_images(self, target_size=19):
        """
        Prepare a square image from a feature vector.
        Args:
            features (numpy array): Feature vector.
            target_size (int): Size of the square image (e.g., 31 for 31x31).

        Returns:
            numpy array: Square image of shape (target_size, target_size).
        """
        total_size = target_size ** 2
        reshaped_data = []  
        with tqdm(total=len(self.data), desc="Creating images", unit="item") as pbar:
            for gesture in self.data:
                gesture_images = []
                for window in gesture:

                    if len(window) > total_size:
                        # Truncate features
                        window = window[:total_size]
                    elif len(window) < total_size:
                        # Pad features with zeros
                        window = np.pad(window, (0, total_size - len(window)), mode='constant')

                    # Normalize to 0-255
                    window_min = np.min(window)
                    window_max = np.max(window)
                    if window_max > window_min:  # Avoid division by zero
                        window = 255 * (window - window_min) / (window_max - window_min)
                    else:
                        window = np.zeros_like(window)  # If constant, set to 0

                    window = window.reshape((target_size, target_size))
                    gesture_images.append(window)
                reshaped_data.append(np.array(gesture_images))
                pbar.update(1)
        self.images = np.array(reshaped_data)

                

    def load_images(self):
        data = []
        labels = []
        for gesture, label in zip(self.images, self.labels):
            for window in gesture:
                data.append(window)
                labels.append(label)

        return np.array(data), np.array(labels, dtype=np.int32)


In [5]:
data_dir = 'data/EMG-EPN612/temp/'  # Replace with the actual path
data_handler = EMGEPN612TrainingDataHandler(data_dir)

Processing Users: 100%|██████████| 50/50 [00:13<00:00,  3.66user/s]


Ground truth available: 7500
No Ground truth available: 0
MINMAX
846
218
max length: 846


Padding Data:  82%|████████▏ | 6118/7500 [00:00<00:00, 15011.28item/s]

TRUNCATED DATA WITH LABEL: 3


Calculating Windows: 100%|██████████| 7500/7500 [00:32<00:00, 231.80item/s]

(7500, 8, 846)
(7500, 7, 352)


In [167]:
print(np.array(data_handler.data).shape)

(7500, 7, 352)


In [6]:
data_handler.create_images()
images = data_handler.images
print(images.shape)

print(images[0])

Creating images: 100%|██████████| 7500/7500 [00:00<00:00, 8759.79item/s]


(7500, 7, 19, 19)
[[[ 15.70925926  15.74855803  15.74074074  15.74074074  14.63888889
    14.60740741  14.51296296  14.67037037  18.88888889  18.88888889
    17.31481481  18.88888889  11.01851852  11.01851852  11.01851852
    11.01851852  15.77222222  15.86666667  15.70925926]
  [ 15.83518519  15.67777778  15.58333333  15.74074074  17.31481481
    15.74074074  17.31481481  15.74074074  14.16666667  15.74074074
    15.74074074  15.74074074  15.74074074  15.74074074  15.74074074
    55.75709548  56.94454767  59.22190318  59.58319578]
  [ 60.85199145  61.36031942  61.98338237  63.42712923  75.21996399
   242.40740741  15.88240741  16.12756204  14.16666667  14.16666667
    14.48148148  14.63888889  14.51296296  14.89074074  23.61111111
    22.03703704  22.03703704  22.03703704   7.87037037]
  [  6.2962963    4.72222222   4.72222222  15.58333333  15.70925926
    15.33148148  15.86666667  15.48888889  15.36296296  17.31481481
    17.31481481  17.31481481  15.74074074  15.74074074  15.7407407

In [7]:
data, labels = data_handler.load_images()
print(data.shape)
print(labels.shape)

(52500, 19, 19)
(52500,)


In [115]:
data, labels = data_handler.load_data()
print("Data shape:", data.shape)
print("Data Length:", data.shape[0])
print("Labels shape:", labels.shape)
print(data[1])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (7500, 8) + inhomogeneous part.

In [4]:
data_dir = 'data/EMG-EPN612/trainingJSON/'  # Replace with the actual path
trainingdata_handler = EMGEPN612TrainingDataHandler(data_dir)


max length: 1112


In [67]:
data, labels = trainingdata_handler.show_data()
print("Data shape:", data.shape)
print("Data Length:", data.shape[0])
print("Labels shape:", labels.shape)
print(labels)

Data shape: (45900, 8, 1112)
Data Length: 45900
Labels shape: (45900,)
['noGesture' 'noGesture' 'noGesture' 'noGesture' 'noGesture' 'noGesture'
 'noGesture' 'noGesture' 'noGesture' 'noGesture' 'noGesture' 'noGesture'
 'noGesture' 'noGesture' 'noGesture' 'noGesture' 'noGesture' 'noGesture'
 'noGesture' 'noGesture' 'noGesture' 'noGesture' 'noGesture' 'noGesture'
 'noGesture' 'fist' 'fist' 'fist' 'fist' 'fist' 'fist' 'fist' 'fist'
 'fist' 'fist' 'fist' 'fist' 'fist' 'fist' 'fist' 'fist' 'fist' 'fist'
 'fist' 'fist' 'fist' 'fist' 'fist' 'fist' 'fist' 'open' 'open' 'open'
 'open' 'open' 'open' 'open' 'open' 'open' 'open' 'open' 'open' 'open'
 'open' 'open' 'open' 'open' 'open' 'open' 'open' 'open' 'open' 'open'
 'open' 'open' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch'
 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch'
 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch' 'pinch'
 'waveIn' 'waveIn' 'waveIn' 'waveIn' 'waveIn' 'waveIn' 'waveIn' 'wa

## CNN

In [8]:
import torch
torch.cuda.is_available()

True

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np

# Mock EMG Dataset
class EMGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # Add height dimension
        self.y = torch.tensor(y, dtype=torch.long)
        print(self.X.shape)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]



# Create Dataset and DataLoader
data, labels = data_handler.load_images()
print(data.shape)
print(labels.shape)
print(data.dtype)
print(labels.dtype)
print(labels)
dataset = EMGDataset(data, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


(52500, 19, 19)
(52500,)
float64
int32
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 

In [ ]:

# Define CNN Model
class EMGCNN(nn.Module):
    def __init__(self, input_channels=1, num_classes=6):
        """
        CNN architecture adjusted for 19x19 input dimensions.
        Args:
            input_channels (int): Number of input channels (default: 1 for grayscale).
            num_classes (int): Number of output classes (default: 6).
        """
        super(EMGCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)  # Output: 32 x 19 x 19
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)              # Output: 64 x 19 x 19
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)                     # Output: 64 x 9 x 9
        self.dropout1 = nn.Dropout(0.25)

        # Fully connected layers
        # Flattened size after pooling: 64 * 9 * 9
        self.fc1 = nn.Linear(64 * 9 * 9, 512)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        """
        Forward pass of the CNN.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, channels, height, width).
        Returns:
            torch.Tensor: Output logits of shape (batch_size, num_classes).
        """
        # Convolution + ReLU + Pooling
        #print(x.shape)
        x = torch.relu(self.conv1(x))  # Shape: (batch_size, 32, 19, 19)
        x = torch.relu(self.conv2(x))  # Shape: (batch_size, 64, 19, 19)
        x = self.pool(x)               # Shape: (batch_size, 64, 9, 9)
        x = self.dropout1(x)

        # Flatten
        x = x.view(x.size(0), -1)      # Shape: (batch_size, 64 * 9 * 9)

        # Fully connected layers + Dropout
        x = torch.relu(self.fc1(x))   # Shape: (batch_size, 512)
        x = self.dropout2(x)
        x = self.fc2(x)               # Shape: (batch_size, num_classes)

        return x

    def train_model(self, train_loader, criterion, optimizer, epochs, device):
        self.to(device)  # Move model to device
        self.train()
        with tqdm(total=epochs, desc="Training Epochs", unit="epoch") as pbar:
            for epoch in range(epochs):
                running_loss = 0.0
                for inputs, labels in train_loader:
                    #print(inputs.shape)
                    inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
                    optimizer.zero_grad()
                    outputs = self(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                tqdm.write(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")
                pbar.update(1)

    def test_model(self, test_loader, device):
        self.to(device)  # Move model to device
        self.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
                outputs = self(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f"Test Accuracy: {100 * correct / total:.2f}%")


# Initialize model, loss function, and optimizer
model = EMGCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train and Evaluate the Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.train_model(train_loader, criterion, optimizer, epochs=50, device=device)
model.test_model(test_loader, device)

# Save the model
# torch.save(model.state_dict() , './models/cnn/emg_cnn_model.pth')

Training Epochs:   2%|▏         | 1/50 [00:02<01:58,  2.42s/epoch]

Epoch 1/50, Loss: 1.5433


Training Epochs:   4%|▍         | 2/50 [00:04<01:48,  2.26s/epoch]

Epoch 2/50, Loss: 1.3381


Training Epochs:   6%|▌         | 3/50 [00:06<01:42,  2.17s/epoch]

...

Epoch 48/50, Loss: 0.9710


Training Epochs:  98%|█████████▊| 49/50 [01:40<00:02,  2.05s/epoch]

Epoch 49/50, Loss: 0.9636


Training Epochs: 100%|██████████| 50/50 [01:42<00:00,  2.05s/epoch]

Epoch 50/50, Loss: 0.9586
Test Accuracy: 64.60%


## EMG DATAHANDLER EXPLO

In [28]:
from libemg.data_handler import OfflineDataHandler
import os
import json
import numpy as np
import torch
import pickle
from torch.utils.data import Dataset
from tqdm import tqdm
from scipy.fftpack import fft

class EMGEPN612TrainingDataHandler_Memmap(OfflineDataHandler):
    def __init__(self, data_dir, memmap_dir, create_data=True):
        """
        Initialize the data handler.

        Args:
            data_dir (str): Directory containing training data.
            memmap_dir (str): Directory to store memory-mapped files.
        """
        super().__init__()
        self.data_dir = data_dir
        self.memmap_dir = memmap_dir

        if create_data:
            metadata = {
                "data_shape": (0, 0, 0),
                "feature_shape": (0, 0, 0),
                "image_shape": (0, 0, 0, 0),
                "labels_shape": (0,),
                "window_metadata": {},
                "dtype": "float32"
            }
            self.update_metadata(metadata)

            # Placeholder attributes to compute dimensions
            self.total_samples = 0
            self.max_length = 0
            self.min_length = float('inf')
            self.sample_data_cache = []  # Temporary storage during `get_data`

            self.get_data()  # First pass to calculate shape
            print(self.max_length)
            print(self.min_length)
            #self.init_memmap_files()  # Initialize memmaps with the correct dimensions
            self.load_data_into_memmap()  # Populate memmap with actual data
            #self.pad_all_data_reflective()
            self.extract_feature_windows()
            self.create_images()
        else:
            self.load_data()

    def update_metadata(self, metadata):
        metadata_path = os.path.join(self.memmap_dir, "metadata.json")
        with open(metadata_path, "w") as f:
            json.dump(metadata, f)

    def get_metadata(self):
        metadata_path = os.path.join(self.memmap_dir, "metadata.json")
        with open(metadata_path, "r") as f:
            metadata = json.load(f)
        return metadata

    def get_data(self):
        """
        First pass to gather dataset statistics and cache sample data.
        """
        gesture = 0
        noGesture = 0
        max_numbers = len(os.listdir(self.data_dir))
        with tqdm(total=max_numbers, desc="Processing Users", unit="user") as pbar:
            for user_folder in os.listdir(self.data_dir):
                user_path = os.path.join(self.data_dir, user_folder)
                if os.path.isdir(user_path):
                    json_file = os.path.join(user_path, f"{user_folder}.json")
                    if os.path.isfile(json_file):
                        with open(json_file, 'r', encoding='utf-8') as f:
                            user_data = json.load(f)
                            if "trainingSamples" in user_data:
                                for sample_key, sample_data in user_data["trainingSamples"].items():
                                    if "gestureName" in sample_data:
                                        gesture += 1
                                        startingPoint = 0
                                        if sample_data["gestureName"] != "noGesture":
                                            startingPoint = sample_data["startPointforGestureExecution"]
                                        emg_data = np.array([
                                            sample_data["emg"]["ch1"][startingPoint:],
                                            sample_data["emg"]["ch2"][startingPoint:],
                                            sample_data["emg"]["ch3"][startingPoint:],
                                            sample_data["emg"]["ch4"][startingPoint:],
                                            sample_data["emg"]["ch5"][startingPoint:],
                                            sample_data["emg"]["ch6"][startingPoint:],
                                            sample_data["emg"]["ch7"][startingPoint:],
                                            sample_data["emg"]["ch8"][startingPoint:]
                                        ])
                                        sample_length = len(emg_data[0])
                                        self.max_length = max(self.max_length, sample_length)
                                        self.min_length = min(self.min_length, sample_length)
                                        if (sample_length <= 0) :
                                            print(sample_data["emg"])
                                            print(sample_data["gestureName"])
                                        
                                        # Cache sample for later use
                                        label = -1
                                        match sample_data["gestureName"]:
                                            case "noGesture": label = 0
                                            case "fist": label = 1
                                            case "waveIn": label = 2
                                            case "waveOut": label = 3
                                            case "open": label = 4
                                            case "pinch": label = 5
                                        self.sample_data_cache.append((emg_data, label))
                                        print(emg_data.shape)
                                        self.total_samples += 1
                                    else:
                                        noGesture += 1
                pbar.update(1)
        print("Ground truth available:", gesture)
        print("No Ground truth available:", noGesture)
        print("Total samples:", self.total_samples)
        print("Max length:", self.max_length)

    def init_memmap_files(self):
        """
        Initialize memory-mapped files based on calculated dimensions.
        """
        os.makedirs(self.memmap_dir, exist_ok=True)
        self.data = np.memmap(
            os.path.join(self.memmap_dir, "data.memmap"), 
            dtype='float32', mode='w+', shape=(self.total_samples, 8, self.max_length)
        )
        self.labels = np.memmap(
            os.path.join(self.memmap_dir, "labels.memmap"), 
            dtype='int32', mode='w+', shape=(self.total_samples,)
        )

        metadata = self.get_metadata()
        metadata["data_shape"] = (self.total_samples, 8, self.max_length)
        metadata["labels_shape"] = (self.total_samples,)
        self.update_metadata(metadata)

        print("Initialized memmap files.")

    def load_data_into_memmap(self):
        """
        Second pass: Save raw EMG data into a flat sequential memory-mapped array.
        """
        # Calculate the total number of samples across all gestures
        total_samples = sum(emg_data.size for emg_data, _ in self.sample_data_cache)
        print("Total samples: " + str(total_samples))
        data_memmap_path = os.path.join(self.memmap_dir, "data.memmap")
        self.data = np.memmap(
            data_memmap_path,
            dtype='float32',
            mode='w+',
            shape=(total_samples,)
        )
        self.labels = np.memmap(
            os.path.join(self.memmap_dir, "labels.memmap"),
            dtype='int32',
            mode='w+',
            shape=(self.total_samples,)
        )

        metadata = self.get_metadata()
        metadata["data_metadata"] = {}
        metadata["data_shape"] = (total_samples,)
        metadata["labels_shape"] = (self.total_samples,)
        current_index = 0

        with tqdm(total=self.total_samples, desc="Loading Data", unit="sample") as pbar:
            for i, (emg_data, label) in enumerate(self.sample_data_cache):
                num_channels, num_samples = emg_data.shape
                flattened_data = emg_data.flatten()  # Flatten data (channels first)
                length = len(flattened_data)

                # Save metadata for this gesture
                metadata["data_metadata"][i] = {
                    "start_index": current_index,
                    "length": length,
                    "num_channels": num_channels,
                    "num_samples": num_samples
                }
                # Save the flattened data
                self.data[current_index:current_index + length] = flattened_data
                self.labels[i] = label

                # Update index
                current_index += length
                pbar.update(1)
        self.update_metadata(metadata)
        self.sample_data_cache = []

    # TODO: try with 0.025, 0 and 0.385, 0.125
    def extract_feature_windows(self, fs=200, window_size=1.0, overlap=0.5):
    #def extract_feature_windows(self, fs=200, window_size=2.0, overlap=0.5):
        """
        Extract features using sliding windows and store in `data` memmap.
        """
        window_samples = int(window_size * fs)
        step_samples = int(window_samples * (1 - overlap))
        feature_size = 352  # Adjust based on the number of features per window
        print(self.data.shape)

        metadata = self.get_metadata()
        data_metadata = metadata["data_metadata"]

        window_metadata = {}
        total_windows = 0

        with tqdm(total=self.total_samples, desc="Calculating Total Windows", unit="gesture") as pbar:
            for i in range(self.total_samples):
                start_index = data_metadata[str(i)]["start_index"]
                length = data_metadata[str(i)]["length"]
                num_samples = length // metadata["data_metadata"][str(i)]["num_channels"] 
                #print(num_samples)
                num_windows = max(0, (num_samples - window_samples) // step_samples + 1)
                window_metadata[i] = {"start_index": total_windows, "num_windows": num_windows}
                total_windows += num_windows
                pbar.update(1)
        print(f"Total windows to be saved: {total_windows}")

        feature_memmap_path = os.path.join(self.memmap_dir, "features.memmap")
        features = np.memmap(
            feature_memmap_path,
            dtype='float32',
            mode='w+',
            shape=(total_windows, feature_size)
        )

        metadata["window_metadata"] = window_metadata
        metadata["feature_shape"] = (total_windows, feature_size)
        self.update_metadata(metadata)

        print(self.data.shape)
        current_window_index = 0
        fixed_bins = np.linspace(0, fs / 2, 10)  # Predefined 10 frequency bins
        with tqdm(total=self.total_samples, desc="Calculating Windows", unit="item") as pbar:
            for i in range(self.total_samples):
                start_index = data_metadata[str(i)]["start_index"]
                length = data_metadata[str(i)]["length"]
                num_channels = metadata["data_metadata"][str(i)]["num_channels"]
                num_samples = length // num_channels
                flattened_data = self.data[start_index:start_index + length]
                emg_data = flattened_data.reshape((num_channels, num_samples))  

                num_windows = window_metadata[i]["num_windows"]
                
                for j in range(num_windows):
                    start = j * step_samples
                    end = start + window_samples
                    segment = emg_data[:, start:end]
                    # Extract features here and write back to memmap
                    window_features = []
                    for channel in range(num_channels):
                        channel_segment = segment[channel, :]

                        half1, half2 = np.split(channel_segment, 2)
                        quarter1, quarter2, quarter3, quarter4 = np.array_split(channel_segment, 4)

                        changes = [
                            np.mean(half2) - np.mean(half1),
                            np.std(half2) - np.std(half1),
                            np.max(half2) - np.max(half1),
                            np.min(half2) - np.min(half1)
                        ]

                        means = [np.mean(quarter) for quarter in [quarter1, quarter2, quarter3, quarter4]]
                        maxs = [np.max(quarter) for quarter in [quarter1, quarter2, quarter3, quarter4]]
                        mins = [np.min(quarter) for quarter in [quarter1, quarter2, quarter3, quarter4]]

                        pairwise_differences = [
                            means[i] - means[j] for i in range(len(means)) for j in range(i + 1, len(means))
                        ] + [
                            maxs[i] - maxs[j] for i in range(len(maxs)) for j in range(i + 1, len(maxs))
                        ] + [
                            mins[i] - mins[j] for i in range(len(mins)) for j in range(i + 1, len(mins))
                        ]

                        fft_magnitudes = np.abs(fft(channel_segment))[:len(channel_segment) // 2]
                        freqs = np.linspace(0, fs / 2, len(fft_magnitudes))  # Actual frequencies
                        interpolated_fft = np.interp(fixed_bins, freqs, fft_magnitudes)

                        # Combine features for this channel
                        channel_features = changes + means + maxs + mins + pairwise_differences + list(interpolated_fft)

                        window_features.extend(channel_features)
                    features[current_window_index, :] = np.array(window_features)
                    current_window_index += 1
                pbar.update(1)

    

    def create_images(self, target_size=19):
        """
        Prepare memory-mapped storage for images and save processed images.
        """
        metadata = self.get_metadata()
        total_size = target_size ** 2
        feature_memmap_path = os.path.join(self.memmap_dir, "features.memmap")
        features = np.memmap(
            feature_memmap_path,
            dtype='float32',
            mode='r',  # Read-only mode to access features
            shape=tuple(metadata["feature_shape"])  # Adjust based on the actual features.memmap shape
        )

        # Calculate number of windows per gesture
        total_windows, feature_size = features.shape

        # Create a memory-mapped file for images
        image_memmap_path = os.path.join(self.memmap_dir, "images.memmap")
        self.images = np.memmap(
            image_memmap_path,
            dtype='float32',
            mode='w+', 
            shape=(total_windows, target_size, target_size)
        )
        metadata["image_shape"] = (total_windows, target_size, target_size)
        self.update_metadata(metadata)

        with tqdm(total=total_windows, desc="Creating Images", unit="gesture") as pbar:
            for window_idx in range(total_windows):
                window = features[window_idx, :]


                # Ensure the feature vector fits the total size of the target image
                if len(window) > total_size:
                    # Truncate features
                    window = window[:total_size]
                elif len(window) < total_size:
                    # Pad features with zeros
                    window = np.pad(window, (0, total_size - len(window)), mode='constant')

                # Normalize the feature vector to the range [0, 255]
                window_min = np.min(window)
                window_max = np.max(window)
                if window_max > window_min:  # Avoid division by zero
                    window = 255 * (window - window_min) / (window_max - window_min)
                else:
                    window = np.zeros_like(window)  # If constant, set to 0

                # Reshape the feature vector into a square image
                window_image = window.reshape((target_size, target_size))
                self.images[window_idx, :, :] = window_image

                # Write the gesture's images to the memory-mapped file
                # self.images[i, :len(gesture_images), :, :] = gesture_images
                pbar.update(1)

        print("Image creation complete.")

    # def load_data(self):
    #     metadata = self.get_metadata()
    #     image_memmap_path = os.path.join(self.memmap_dir, "images.memmap")
    #     labels_memmap_path = os.path.join(self.memmap_dir, "labels.memmap")
    #     features_memmap_path = os.path.join(self.memmap_dir, "features.memmap")
    #     data_memmap_path = os.path.join(self.memmap_dir, "data.memmap")

    #     self.data = np.memmap(
    #         data_memmap_path,
    #         dtype='float32',
    #         mode='r', 
    #         shape=tuple(metadata["data_shape"])
    #     )
    #     print(self.data.shape)    
    #     features = np.memmap(
    #         features_memmap_path,
    #         dtype='float32',
    #         mode='r', 
    #         shape=tuple(metadata["feature_shape"])
    #     )
    #     print(features.shape)
    #     self.images = np.memmap(
    #         image_memmap_path,
    #         dtype='float32',
    #         mode='r', 
    #         shape=tuple(metadata["image_shape"])
    #     )
    #     print(self.images.shape)
    #     self.labels = np.memmap(
    #         labels_memmap_path,
    #         dtype='float32',
    #         mode='r', 
    #         shape=tuple(metadata["labels_shape"])
    #     )
    #     print(self.labels.shape)

    #     print(metadata["window_metadata"])
    
    def load_data(self):
        metadata = self.get_metadata()
        image_memmap_path = os.path.join(self.memmap_dir, "images.memmap")
        labels_memmap_path = os.path.join(self.memmap_dir, "labels.memmap")
        image_shape = tuple(metadata["image_shape"])
        label_shape = tuple(metadata["labels_shape"])
        window_metadata = metadata["window_metadata"]

        return image_memmap_path, labels_memmap_path, image_shape, label_shape, window_metadata

In [ ]:
TRAINING_DIR = 'data/EMG-EPN612/temp/' 
#TESTING_DIR = 'data/EMG-EPN612/testingJSON/'
DATA_HANDLER_DIR = 'handler/'

training_data_handler = EMGEPN612TrainingDataHandler_Memmap(TRAINING_DIR, DATA_HANDLER_DIR + '612training_test', True)


Processing Users:   2%|▏         | 1/50 [00:00<00:15,  3.10user/s]

(8, 996)
(8, 992)
(8, 992)
(8, 998)
(8, 992)
(8, 998)
(8, 996)
(8, 994)
(8, 996)
(8, 996)
(8, 992)
(8, 992)
(8, 1000)
(8, 996)
(8, 998)
(8, 997)
(8, 996)
(8, 998)
(8, 996)
(8, 1000)
(8, 996)
(8, 996)
(8, 994)
(8, 992)
(8, 992)
(8, 371)
(8, 450)
(8, 599)
(8, 481)
(8, 683)
(8, 767)
(8, 369)
(8, 437)
(8, 459)
(8, 650)
(8, 384)
(8, 503)
(8, 793)
(8, 749)
(8, 434)
(8, 467)
(8, 448)
(8, 395)
(8, 448)
(8, 708)
(8, 364)
(8, 573)
(8, 655)
(8, 707)
(8, 371)
(8, 365)
(8, 825)
(8, 438)
(8, 469)
(8, 655)
(8, 764)
(8, 652)
(8, 685)
(8, 779)
(8, 435)
(8, 446)
(8, 623)
(8, 379)
(8, 753)
(8, 828)
(8, 828)
(8, 493)
(8, 635)
(8, 389)
(8, 707)
(8, 425)
(8, 505)
(8, 803)
(8, 706)
(8, 366)
(8, 448)
(8, 499)
(8, 434)
(8, 463)
(8, 627)
(8, 399)
(8, 650)
(8, 749)
(8, 371)
(8, 463)
(8, 367)
(8, 444)
(8, 491)
(8, 825)
(8, 756)
(8, 834)
(8, 822)
(8, 631)
(8, 379)
(8, 452)
(8, 763)
(8, 503)
(8, 777)
(8, 703)
(8, 497)
(8, 368)
(8, 450)
(8, 495)
(8, 827)
(8, 603)
(8, 703)
(8, 474)
(8, 435)
(8, 785)
(8, 683)
(8, 603)

Processing Users:   4%|▍         | 2/50 [00:00<00:14,  3.28user/s]

(8, 996)
(8, 995)
(8, 1000)
(8, 992)
(8, 996)
(8, 996)
(8, 996)
(8, 994)
(8, 992)
(8, 998)
(8, 996)
(8, 994)
(8, 996)
(8, 996)
(8, 995)
(8, 992)
(8, 998)
(8, 996)
(8, 996)
(8, 996)
(8, 996)
(8, 996)
(8, 996)
(8, 996)
(8, 998)
(8, 799)
(8, 573)
(8, 828)
(8, 366)
(8, 478)
(8, 518)
(8, 469)
(8, 525)
(8, 489)
(8, 467)
(8, 467)
(8, 450)
(8, 657)
(8, 689)
(8, 435)
(8, 375)
(8, 446)
(8, 710)
(8, 599)
(8, 367)
(8, 367)
(8, 601)
(8, 391)
(8, 448)
(8, 395)
(8, 830)
(8, 520)
(8, 707)
(8, 767)
(8, 525)
(8, 452)
(8, 395)
(8, 704)
(8, 823)
(8, 797)
(8, 499)
(8, 759)
(8, 417)
(8, 683)
(8, 653)
(8, 797)
(8, 442)
(8, 631)
(8, 367)
(8, 437)
(8, 633)
(8, 361)
(8, 467)
(8, 507)
(8, 463)
(8, 767)
(8, 391)
(8, 367)
(8, 519)
(8, 621)
(8, 364)
(8, 444)
(8, 514)
(8, 635)
(8, 801)
(8, 714)
(8, 363)
(8, 779)
(8, 824)
(8, 470)
(8, 687)
(8, 756)
(8, 463)
(8, 825)
(8, 389)
(8, 499)
(8, 801)
(8, 437)
(8, 448)
(8, 623)
(8, 523)
(8, 764)
(8, 687)
(8, 771)
(8, 439)
(8, 522)
(8, 832)
(8, 601)
(8, 375)
(8, 708)
(8, 381)


Processing Users:   6%|▌         | 3/50 [00:00<00:13,  3.44user/s]

...

Processing Users:  78%|███████▊  | 39/50 [00:11<00:03,  3.44user/s]

(8, 992)
(8, 996)
(8, 992)
(8, 994)
(8, 996)
(8, 996)
(8, 996)
(8, 998)
(8, 992)
(8, 996)
(8, 996)
(8, 996)
(8, 992)
(8, 998)
(8, 996)
(8, 996)
(8, 992)
(8, 992)
(8, 996)
(8, 992)
(8, 994)
(8, 992)
(8, 992)
(8, 992)
(8, 994)
(8, 595)
(8, 446)
(8, 439)
(8, 463)
(8, 476)
(8, 793)
(8, 775)
(8, 704)
(8, 489)
(8, 830)
(8, 823)
(8, 384)
(8, 829)
(8, 434)
(8, 503)
(8, 619)
(8, 448)
(8, 655)
(8, 463)
(8, 371)
(8, 683)
(8, 389)
(8, 648)
(8, 360)
(8, 623)
(8, 771)
(8, 707)
(8, 363)
(8, 446)
(8, 446)
(8, 393)
(8, 749)
(8, 434)
(8, 621)
(8, 435)
(8, 474)
(8, 499)
(8, 371)
(8, 648)
(8, 434)
(8, 530)
(8, 518)
(8, 435)
(8, 826)
(8, 828)
(8, 619)
(8, 651)
(8, 450)
(8, 459)
(8, 417)
(8, 783)
(8, 779)
(8, 367)
(8, 435)
(8, 371)
(8, 534)
(8, 423)
(8, 363)
(8, 467)
(8, 625)
(8, 826)
(8, 511)
(8, 650)
(8, 819)
(8, 459)
(8, 707)
(8, 446)
(8, 799)
(8, 487)
(8, 637)
(8, 760)
(8, 706)
(8, 512)
(8, 367)
(8, 450)
(8, 518)
(8, 707)
(8, 704)
(8, 771)
(8, 824)
(8, 601)
(8, 797)
(8, 367)
(8, 461)
(8, 653)
(8, 360)
(

Processing Users:  80%|████████  | 40/50 [00:12<00:02,  3.47user/s]

(8, 998)
(8, 996)
(8, 992)
(8, 996)
(8, 990)
(8, 1002)
(8, 996)
(8, 990)
(8, 994)
(8, 994)
(8, 992)
(8, 992)
(8, 994)
(8, 992)
(8, 996)
(8, 1002)
(8, 996)
(8, 992)
(8, 992)
(8, 996)
(8, 992)
(8, 996)
(8, 996)
(8, 992)
(8, 994)
(8, 497)
(8, 651)
(8, 448)
(8, 528)
(8, 387)
(8, 503)
(8, 599)
(8, 571)
(8, 779)
(8, 430)
(8, 522)
(8, 393)
(8, 499)
(8, 801)
(8, 775)
(8, 648)
(8, 621)
(8, 703)
(8, 439)
(8, 633)
(8, 369)
(8, 459)
(8, 453)
(8, 432)
(8, 395)
(8, 443)
(8, 375)
(8, 707)
(8, 452)
(8, 360)
(8, 769)
(8, 520)
(8, 783)
(8, 834)
(8, 446)
(8, 509)
(8, 423)
(8, 828)
(8, 637)
(8, 490)
(8, 828)
(8, 655)
(8, 687)
(8, 797)
(8, 469)
(8, 519)
(8, 379)
(8, 366)
(8, 495)
(8, 465)
(8, 683)
(8, 635)
(8, 828)
(8, 389)
(8, 832)
(8, 520)
(8, 489)
(8, 829)
(8, 366)
(8, 767)
(8, 436)
(8, 650)
(8, 489)
(8, 467)
(8, 777)
(8, 607)
(8, 756)
(8, 703)
(8, 421)
(8, 503)
(8, 797)
(8, 452)
(8, 619)
(8, 797)
(8, 388)
(8, 518)
(8, 797)
(8, 439)
(8, 599)
(8, 365)
(8, 359)
(8, 423)
(8, 526)
(8, 371)
(8, 444)
(8, 607)

Processing Users:  82%|████████▏ | 41/50 [00:12<00:02,  3.49user/s]

## Grabmyo

In [15]:
import scipy.io
mat = scipy.io.loadmat('data/grabmyo/Output BM/Session1_converted/session1_participant1.mat')
mat['DATA_FOREARM'].shape
# (repetition, gesture)

(7, 17)

In [ ]:
mat['DATA_FOREARM'][0][0].shape
# (2048*5, channels)
#mat['DATA_FOREARM'][0][0]

(10240, 16)

In [7]:
from scipy.signal import resample
import scipy.io
import numpy as np
resample_length = 200 * 5  # downsample to 200 hz


N_classes = 17
N_repetitions = 7
N_sessions = 1
N_subjects = 41
labels = []
class_filter = [10, 11, 14, 15, 16]

for session in range(1, N_sessions+1):
    for sub in range(1, N_subjects+1):
        filepath = 'data/grabmyo/Output BM/Session{}_converted/'.format(session) + 'session{}_participant{}.mat'.format(session, sub)
        mat = scipy.io.loadmat(filepath)  
        mat = mat['DATA_FOREARM']  
        for repetition in range(0, N_repetitions):
            for cls in range(0, N_classes):
                if cls in class_filter:
                    emg_data = mat[repetition][cls]
                    emg_data = np.nan_to_num(emg_data)
                    emg_data = resample(emg_data, resample_length)
                    emg_data = emg_data.T
                    label = -1
                    match cls:
                        case 16: label = 0
                        case 15: label = 1
                        case 11: label = 2
                        case 10: label = 3
                        case 14: label = 4

                    # print(emg_data.shape)

                    self.sample_data_cache.append((emg_data, label))
                    self.total_samples += 1


len(labels)

NameError: name 'self' is not defined

In [17]:
labels

[10,
 11,
 14,
 15,
 10,
 11,
 14,
 15,
 10,
 11,
 14,
 15,
 10,
 11,
 14,
 15,
 10,
 11,
 14,
 15,
 10,
 11,
 14,
 15,
 10,
 11,
 14,
 15]